In [1]:
import numpy as np
import pickle
from typing import List
import openflash
print(type(openflash))
print(openflash.__path__)
print(openflash.__file__)

# --- Import core modules from package ---
try:
    from openflash import *
    from openflash.multi_equations import *
    from openflash.multi_constants import g
    print("OpenFLASH modules imported successfully!")
except ImportError as e:
    print(f"Error importing OpenFLASH modules. Error: {e}")

# Set NumPy print options for better readability
np.set_printoptions(threshold=np.inf, linewidth=np.inf, precision=8, suppress=True)
from openflash.multi_constants import rho #need for BEM
from openflash.multi_equations import wavenumber

from capytaine_generator import CapytaineSlantSolver
import capytaine as cpt

# Now you can import from the folder structure
# Assuming 'pyplotutilities' is a folder inside 'sea-lab-utils'
import sys
from pathlib import Path
HERE = Path.cwd().resolve()
store_path_prefix = str((HERE / "data").resolve())
store_path_str = store_path_prefix + "/slants.pkl"

multi_condensed_dir = (HERE / ".." / "dev" / "python").resolve()
if str(multi_condensed_dir) not in sys.path:
    sys.path.insert(0, str(multi_condensed_dir))

# slant_helpers = (HERE / ".." / "dev" / "python" / "slants").resolve()
# if str(slant_helpers) not in sys.path:
#     sys.path.insert(0, str(slant_helpers))
# from helpers import solve_problem, slant_approx_vars, make_slant_region2, update_data_file

modified_meem_dir = (HERE / ".." / "dev" / "python" / "slants" / "a-matrix-b-vector-changes").resolve()
if str(modified_meem_dir) not in sys.path:
    sys.path.insert(0, str(modified_meem_dir))
from modified_meem import SProblem, solve_modified_problem, slant_approx_vars, update_data_file

<class 'module'>
['/Users/Bimali/Desktop/SEALab/OpenFLASH/package/src/openflash']
/Users/Bimali/Desktop/SEALab/OpenFLASH/package/src/openflash/__init__.py
OpenFLASH modules imported successfully!


In [6]:
# Plot 1: Use the Cor-Power WEC outline.
# Plot 2: Same cross section and max depth, but a cone.
# Plot 3: Same cross section but significantly shorter max depth (cone)
# Plot 4: Same cross section but significantly taller max depth (cone)

h = 50.00 # sea depth [m]
omega_sweep = np.linspace(0.4, 1.5, 10) # rad/s
omega_fixed = 1
big_nmk = 150

d_in_1 = [14.45, 14.45-7.32] # drafts [m]
d_out_1 = [14.45, 14.45-7.32-5.08] # non-slanted version
a_list_1 = [2.5/2, 8.4/2] # radii [m]
NMK_1 = [big_nmk, big_nmk, big_nmk] # number of coefficients in each region

d_in_2 = [14.45]
d_out_2 = [1e-4]
a_list_2 = [8.4/2]
NMK_2 = [big_nmk, big_nmk]

d_in_3 = [1.26]
d_out_3 = [1e-4]
a_list_3 = [8.4/2]
NMK_3 = [big_nmk, big_nmk]

d_in_4 = [40]
d_out_4 = [1e-4]
a_list_4 = [8.4/2]
NMK_4 = [big_nmk, big_nmk]

res_lst = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 25, 30]

In [7]:
css = CapytaineSlantSolver(False, False, False, False, False)
def compute_cpt_slant(config, t_densities):
  respack = css.construct_and_solve(config["a"], config["d_in"], config["d_out"], config["heaving"], t_densities, 150, config["h"], config["omega"], config["rho"], 1)
  am, dp, f_ex = (respack[0].added_mass)["Heave"], (respack[0].radiation_damping)["Heave"], (cpt.assemble_dataset([respack[2]]))["excitation_force"][0][0][0].values
  return am, dp, f_ex

def BEM_hydro_coeffs(h, rho, omega_sweep, d_in, d_out, a_list, t_densities):
    # Run BEM
    heaving_list = [1 for _ in range(len(a_list))]
    A_BEM = np.zeros_like(omega_sweep)
    B_BEM = np.zeros_like(omega_sweep)
    F_ex_BEM = np.zeros(len(omega_sweep), dtype=np.complex128)

    for j in range(len(omega_sweep)):
        i=1
        omega = omega_sweep[j]
        config = {"name" : "config-(" + str(i) + ")",
                    "h" : h,
                    "a" : a_list,
                    "d_in" : d_in,
                    "d_out" : d_out,
                    "heaving": heaving_list,
                    "NMK" : [100, 100], #this is unused
                    "omega" : omega,
                    "rho" : rho}
        Capy_results = compute_cpt_slant(config, t_densities)
        A_BEM[j] = Capy_results[0]
        B_BEM[j] = Capy_results[1]
        F_ex_BEM[j] = Capy_results[2]
    return A_BEM, B_BEM, F_ex_BEM

In [ ]:
def get_MEEM_hydros(h, omega_sweep, d_list, a_list, NMK, heaving):

    A_MEEM = np.zeros_like(omega_sweep)
    B_MEEM = np.zeros_like(omega_sweep)

    for j in range(len(omega_sweep)):

        # 1. Create SteppedBody objects
        bodies_sweep = []

        m0 = wavenumber(omega_sweep[j], h)
        
        # Single Body
        body = SteppedBody(a=np.array(a_list), d=np.array(d_list), slant_angle= np.zeros_like(a_list), heaving=heaving)
        bodies_sweep.append(body)
        
        # 2. Create arrangement
        arrangement_sweep = ConcentricBodyGroup(bodies_sweep)

        # 3. Create geometry
        geometry_sweep = BasicRegionGeometry(body_arrangement=arrangement_sweep, h=h, NMK=NMK)

        # 4. Create the MEEMProblem instance
        problem = MEEMProblem(geometry_sweep)

        # 5. Set the frequencies for the sweep
        problem.set_frequencies(np.array([omega_sweep[j]]))

        # 6. Initialize a new MEEM Engine for this problem
        engine = MEEMEngine(problem_list=[problem])

        # 7. Solve
        X = engine.solve_linear_system_multi(problem, m0)

        # 8. Coefficients
        coeffs = engine.compute_hydrodynamic_coefficients(problem, X, m0)
        # print(coeffs[0])
        
        A_MEEM[j] = coeffs[0]['real']
        B_MEEM[j] = coeffs[0]['imag']
        print(f"Finished an omega value: {omega_sweep[j]:.3f}")

    return A_MEEM, B_MEEM

def get_unmodified_MEEM_hydros(h, omega_sweep, d_in, d_out, a_list, NMK, heaving, res):
    d_prime, a_prime, heaving_prime, NMK_prime, slopes, d_in_prime, d_out_prime = slant_approx_vars(a_list, d_in, d_out, heaving, NMK, res, 2)
    A_MEEM, B_MEEM = get_MEEM_hydros(h, omega_sweep, d_prime, a_prime, NMK_prime, True)
    return A_MEEM, B_MEEM

def get_modified_MEEM_hydros(h, omega_sweep, d_in, d_out, a_list, NMK, heaving, res, version = 2, frac1 = 0.5, frac2 = None):
    def outline_function(r):
      region = 0
      for rad in a_list:
        if r <= rad: break
        else: region += 1
      if region == len(a_list): return 0
      elif d_in[region] == d_out[region]:
        return - d_in[region]
      else:
        inner_rad = 0 if region == 0 else a_list[region - 1]
        outer_rad = a_list[region]
        slope = (d_in[region] - d_out[region])/(outer_rad - inner_rad)
        return slope * (r - inner_rad) - d_in[region]
      
    A_MEEM, B_MEEM = [], []
    for omega_val in omega_sweep:
      m0 = wavenumber(omega_val, h)
      x, cs, prob = solve_modified_problem(h, a_list, d_in, d_out, heaving, m0, rho, res, version, NMK = NMK)
      am, dp = prob.hydros_by_averages(cs, outline_function, "capytaine", frac1 = frac1, frac2 = frac2)
      A_MEEM.append(am)
      B_MEEM.append(dp)
      print(f"Finished an omega value: {omega_val:.3f}")
    return np.array(A_MEEM), np.array(B_MEEM)

In [5]:
data = {"Config 1 Sweep" : {},
        "Config 1" : {},
        "Config 2" : {},
        "Config 3" : {},
        "Config 4" : {}}

In [ ]:
with open(store_path_str, 'rb') as file:
    data = pickle.load(file)

# A_BEM, B_BEM, F_ex_BEM = BEM_hydro_coeffs(h, rho, omega_sweep, d_in_1, d_out_1, a_list_1, [30, 50])
# data["Config 1 Sweep"]["CPT Added Mass"] = A_BEM
# data["Config 1 Sweep"]["CPT Damping"] = B_BEM
# data["Config 1 Sweep"]["CPT Exc. Force"] = F_ex_BEM
# update_data_file(data, store_path_str)

# A_MEEM_unmod, B_MEEM_unmod = get_unmodified_MEEM_hydros(h, omega_sweep, d_in_1, d_out_1, a_list_1, NMK_1, [1, 1], 30)
# data["Config 1 Sweep"]["MEEM Added Mass"] = A_MEEM_unmod
# data["Config 1 Sweep"]["MEEM Damping"] = B_MEEM_unmod
# update_data_file(data, store_path_str)

# A_MEEM_mod, B_MEEM_mod = get_modified_MEEM_hydros(h, omega_sweep, d_in_1, d_out_1, a_list_1, NMK_1, [1, 1], 30)
# data["Config 1 Sweep"]["Modified MEEM Added Mass"] = A_MEEM_mod
# data["Config 1 Sweep"]["Modified MEEM Damping"] = B_MEEM_mod
# update_data_file(data, store_path_str)

Finished an omega value: 0.400
Finished an omega value: 0.522
Finished an omega value: 0.644
Finished an omega value: 0.767
Finished an omega value: 0.889
Finished an omega value: 1.011
Finished an omega value: 1.133
Finished an omega value: 1.256
Finished an omega value: 1.378
Finished an omega value: 1.500
Finished an omega value: 0.400
Finished an omega value: 0.522
Finished an omega value: 0.644
Finished an omega value: 0.767
Finished an omega value: 0.889
Finished an omega value: 1.011
Finished an omega value: 1.133
Finished an omega value: 1.256
Finished an omega value: 1.378
Finished an omega value: 1.500


In [7]:
config_geoms = {"Config 1" : {"d_in" : d_in_1, "d_out" : d_out_1, "a_list" : a_list_1, "NMK" : NMK_1},
                "Config 2" : {"d_in" : d_in_2, "d_out" : d_out_2, "a_list" : a_list_2, "NMK" : NMK_2},
                "Config 3" : {"d_in" : d_in_3, "d_out" : d_out_3, "a_list" : a_list_3, "NMK" : NMK_3},
                "Config 4" : {"d_in" : d_in_4, "d_out" : d_out_4, "a_list" : a_list_4, "NMK" : NMK_4}}

omega_lst = np.array([omega_fixed])

In [ ]:
# with open(store_path_str, 'rb') as file:
#     data = pickle.load(file)

# for index in [1, 2, 3, 4]:
#   if index == 1: t_densities = [30, 50]
#   else: t_densities = [40]
#   cf_name = "Config " + str(index)
#   heaving = [1 for _ in range(len(config_geoms[cf_name]["d_in"]))]
#   A_BEM, B_BEM, F_ex_BEM = BEM_hydro_coeffs(h, rho, omega_lst,
#                                             config_geoms[cf_name]["d_in"],
#                                             config_geoms[cf_name]["d_out"],
#                                             config_geoms[cf_name]["a_list"], t_densities)
#   data[cf_name]["CPT Added Mass"] = A_BEM[0]
#   data[cf_name]["CPT Damping"] = B_BEM[0]
#   data[cf_name]["CPT Exc. Force"] = F_ex_BEM[0]
# update_data_file(data, store_path_str)

In [9]:
with open(store_path_str, 'rb') as file:
    data = pickle.load(file)

for index in [1, 2, 3, 4]:
  cf_name = "Config " + str(index)
  heaving = [1 for _ in range(len(config_geoms[cf_name]["d_in"]))]
  # data[cf_name]["MEEM Added Mass"] = {}
  # data[cf_name]["MEEM Damping"] = {}
  # data[cf_name]["Modified MEEM Added Mass"] = {}
  # data[cf_name]["Modified MEEM Damping"] = {}
  for res in res_lst:
    A_MEEM_unmod, B_MEEM_unmod = get_unmodified_MEEM_hydros(h, omega_lst,
                                                            config_geoms[cf_name]["d_in"],
                                                            config_geoms[cf_name]["d_out"],
                                                            config_geoms[cf_name]["a_list"],
                                                            config_geoms[cf_name]["NMK"], heaving, res)
    data[cf_name]["MEEM Added Mass"][res] = A_MEEM_unmod[0]
    data[cf_name]["MEEM Damping"][res] = B_MEEM_unmod[0]

    A_MEEM_mod, B_MEEM_mod = get_modified_MEEM_hydros(h, omega_lst,
                                                      config_geoms[cf_name]["d_in"],
                                                      config_geoms[cf_name]["d_out"],
                                                      config_geoms[cf_name]["a_list"],
                                                      config_geoms[cf_name]["NMK"], heaving, res)
    data[cf_name]["Modified MEEM Added Mass"][res] = A_MEEM_mod[0]
    data[cf_name]["Modified MEEM Damping"][res] = B_MEEM_mod[0]
    print("Finished res = " + str(res))
    update_data_file(data, store_path_str)
  print("Finished " + cf_name)

/Users/Bimali/Desktop/SEALab/OpenFLASH/package/src/openflash/meem_engine.py:307: LinAlgWarning: Ill-conditioned matrix (rcond=3.84051e-29): result may not be accurate.
  X = linalg.solve(A, b)


Finished an omega value: 1.000


/Users/Bimali/Desktop/SEALab/OpenFLASH/dev/python/multi_condensed.py:553: LinAlgWarning: Ill-conditioned matrix (rcond=3.84051e-29): result may not be accurate.
  return linalg.solve(a,b)


Finished an omega value: 1.000
Finished res = 1


/Users/Bimali/Desktop/SEALab/OpenFLASH/package/src/openflash/meem_engine.py:307: LinAlgWarning: Ill-conditioned matrix (rcond=6.34888e-23): result may not be accurate.
  X = linalg.solve(A, b)


...

Finished an omega value: 1.000
Finished res = 9


/Users/Bimali/Desktop/SEALab/OpenFLASH/package/src/openflash/meem_engine.py:307: LinAlgWarning: Ill-conditioned matrix (rcond=6.75352e-17): result may not be accurate.
  X = linalg.solve(A, b)


Finished an omega value: 1.000


/Users/Bimali/Desktop/SEALab/OpenFLASH/dev/python/multi_condensed.py:553: LinAlgWarning: Ill-conditioned matrix (rcond=6.03531e-17): result may not be accurate.
  return linalg.solve(a,b)


Finished an omega value: 1.000
Finished res = 10
Finished an omega value: 1.000
Finished an omega value: 1.000
Finished res = 15
Finished an omega value: 1.000
Finished an omega value: 1.000
Finished res = 20
Finished an omega value: 1.000
Finished an omega value: 1.000
Finished res = 25
Finished an omega value: 1.000
Finished an omega value: 1.000
Finished res = 30
Finished Config 4
